### Finviz Data Cleaning and Transformation Pipeline

This notebook processes the raw data downloaded from Finviz, cleans it, and transforms it into an analysis-ready format.

**Workflow:**

1.  **Load Data:** Reads a raw Finviz `.parquet` file for a specific date.
2.  **Feature Engineering:** Creates new composite columns (`Info`, `MktCap AUM`).
3.  **Data Type Conversion:**
    *   Converts currency strings (e.g., `1.5B`, `250K`) into numeric values in millions.
    *   Converts percentage strings (e.g., `12.5%`) into numeric values.
    *   Converts other object columns to their proper numeric types.
4.  **Final Processing:** Sorts the data by market capitalization, sets the `Ticker` as the index, and adds a `Rank` column.
5.  **Save & Verify:** Saves the cleaned DataFrame to a new `.parquet` file and verifies the saved file.

### Setup and Configuration

**This is the only cell you need to modify.** It contains all imports, paths, and lists of columns for processing.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import config and custom utils now that path is set
from config import DATE_STR, DOWNLOAD_DIR, DEST_DIR
import utils

# --- File Path Configuration ---
# Build paths using pathlib for cross-platform compatibility
SOURCE_PATH = Path(DOWNLOAD_DIR) / f'df_finviz_{DATE_STR}_stocks_etfs.parquet'
DEST_PATH = Path(DEST_DIR) / f'{DATE_STR}_df_finviz_stocks_etfs.parquet'

# --- Column Processing Configuration ---
# Define which columns need specific cleaning operations.

# Columns to combine into the 'Info' column
INFO_COLS = ["Sector", "Industry", "Single Category", "Asset Type"]

# Columns with abbreviated currency values (B, M, K) to be converted to millions
CURRENCY_COLS = [
    'Market Cap', 'AUM', 'Sales', 'Income', 'Outstanding', 'Float', 
    'Short Interest', 'Avg Volume', 'Flows 1M', 'Flows 3M', 'Flows YTD',
    'MktCap AUM' # This is the new column we create
]

# Other columns that are numeric but stored as strings (objects)
# Note: Percentage columns are detected automatically in Step 3.
OTHER_NUMERIC_COLS = [
    "No.", "P/E", "Fwd P/E", "PEG", "P/S", "P/B", "P/C", "P/FCF",
    "Book/sh", "Cash/sh", "Dividend TTM", "EPS", "EPS next Q", "Short Ratio",
    "Curr R", "Quick R", "LTDebt/Eq", "Debt/Eq", "Beta", "ATR", "RSI",
    "Employees", "Recom", "Rel Volume", "Volume", "Target Price",
    "Prev Close", "Open", "High", "Low", "Price", "Holdings"
]

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Source file: {SOURCE_PATH}")
print(f"Destination file: {DEST_PATH}")
print(f"Processing for date: {DATE_STR}")

Source file: C:\Users\ping\Downloads\df_finviz_2025-12-12_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks\data\2025-12-12_df_finviz_stocks_etfs.parquet
Processing for date: 2025-12-12


### Step 1: Load Raw Data

Load the source Parquet file into a pandas DataFrame.

In [2]:
print(f"--- Step 1: Loading data from {SOURCE_PATH.name} ---")

try:
    df = pd.read_parquet(SOURCE_PATH, engine='pyarrow')
    print("Data loaded successfully.")
    df.info()
    display(df.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}")
    df = None  # Ensure df is None if loading fails
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df = None

--- Step 1: Loading data from df_finviz_2025-12-12_stocks_etfs.parquet ---
Data loaded successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 111 entries, No. to Tags
dtypes: object(111)
memory usage: 1.3+ MB


,No.,Ticker,Company,Index,Sector,Industry,Country,Exchange,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend,Dividend TTM,Dividend Ex Date,Payout Ratio,EPS,EPS next Q,EPS This Y,EPS Next Y,EPS Past 5Y,EPS Next 5Y,Sales Past 5Y,Sales Q/Q,EPS Q/Q,EPS YoY TTM,Sales YoY TTM,Sales,Income,EPS Surprise,Revenue Surprise,Outstanding,Float,Float %,Insider Own,Insider Trans,Inst Own,Inst Trans,Short Float,Short Ratio,Short Interest,ROA,ROE,ROIC,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,52W Range,All-Time High,All-Time Low,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open,Gap,Recom,Avg Volume,Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change,Single Category,Asset Type,Expense,Holdings,AUM,Flows 1M,Flows% 1M,Flows 3M,Flows% 3M,Flows YTD,Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags
0,841,IVZ,Invesco Ltd,S&P 500,Financial,Asset Management,USA,NYSE,11.69B,17.83,10.37,0.54,1.86,1.06,7.67,9.42,24.70,3.42,3.20%,0.83,11/14/2025,69.23%,1.47,0.57,15.87%,27.77%,-1.69%,19.14%,-0.47%,8.63%,449.13%,260.65%,6.64%,6.29B,669.20M,36.10%,0.91%,445.09M,442.08M,99.32%,0.68%,-8.00%,92.14%,-6.18%,2.89%,2.48,12.76M,3.75%,7.29%,2.99%,0.98,0.98,0.60,0.60,64.11%,17.98%,10.63%,0.08%,6.49%,17.55%,80.73%,42.33%,50.23%,1.63,0.82,3.45%,3.39%,6.80%,9.75%,37.99%,-3.83%,27.04%,-3.83%,126.38%,11.60 - 27.31,-48.61%,318.49%,63.26,Oct 28/b,8/25/1995,Yes,Yes,8508,-3.24%,0.52%,2.60,5.15M,1.09,"5,612,093",27.25,27.00,27.14,27.16,26.16,26.26,-2.74%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,842,RDY,Dr. Reddy's Laboratories Ltd. ADR,-,Healthcare,Drug Manufacturers - Specialty & Generic,India,NYSE,11.67B,17.16,22.26,-,2.93,2.90,14.84,85.17,4.84,0.94,0.66%,0.09,7/25/2025,8.90%,0.82,0.17,-10.93%,-10.95%,19.33%,-1.45%,9.35%,5.38%,9.74%,6.35%,10.63%,3.98B,680.48M,1.92%,2.30%,834.61M,832.46M,99.74%,0.00%,-,13.14%,3.98%,1.11%,8.57,9.21M,11.67%,17.72%,16.32%,1.85,1.35,0.03,0.16,58.70%,19.25%,17.10%,-1.20%,1.23%,-4.17%,-11.27%,-2.09%,-11.21%,0.37,0.21,1.65%,1.31%,0.31%,0.08%,-0.67%,-6.21%,6.70%,-13.30%,14.36%,12.26 - 16.17,-17.00%,1296.41%,51.62,Oct 24/b,4/11/2001,Yes,Yes,27811,-0.50%,0.00%,2.79,1.07M,1.07,"1,154,680",14.89,14.09,14.09,14.16,13.97,14.02,-0.50%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,843,GMED,Globus Medical Inc,-,Healthcare,Medical Devices,USA,NYSE,11.67B,28.02,21.52,1.78,4.21,2.65,29.88,20.13,32.68,2.90,-,-,-,0.00%,3.09,1.07,20.78%,6.24%,-13.26%,12.07%,26.25%,22.91%,134.07%,364.73%,11.75%,2.77B,423.78M,51.13%,4.68%,112.18M,111.38M,99.29%,17.25%,-0.41%,88.30%,3.48%,3.95%,2.55,4.40M,8.32%,10.01%,9.41%,4.13,2.47,0.02,0.03,61.05%,16.20%,15.30%,-4.83%,2.35%,48.87%,47.20%,4.60%,4.82%,1.06,2.42,2.30%,2.69%,-1.03%,17.45%,31.95%,-6.24%,53.45%,-8.67%,67.41%,51.79 - 94.93,-8.67%,745.03%,58.22,Nov 06/a,8/3/2012,Yes,Yes,5300,-2.42%,-0.03%,1.93,1.73M,0.80,"1,389,600",97.60,88.88,88.85,88.87,86.09,86.70,-2.45%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


### Step 2: Feature Engineering - Create Composite Columns

Combine existing columns to create more meaningful features: `Info` and `MktCap AUM`.

In [3]:
if df is not None:
    print("\n--- Step 2: Engineering new features ---")
    
    # 1. Create 'Info' column by concatenating category columns.
    for col in INFO_COLS:
        if col in df.columns:
            df[col] = df[col].replace('-', '')
    df['Info'] = df[INFO_COLS].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)
    print("Created 'Info' column.")

    # 2. Create 'MktCap AUM' by concatenating 'Market Cap' and 'AUM'.
    # This combines stock and ETF liquidity metrics into a single string column for now.
    # It will be converted to numeric in the next step.
    df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')
    print("Created 'MktCap AUM' column.")

    # Display the new columns for verification
    display(df[['Ticker', 'Info', 'MktCap AUM']].head(3))


--- Step 2: Engineering new features ---
Created 'Info' column.
Created 'MktCap AUM' column.


,Ticker,Info,MktCap AUM
0,IVZ,"Financial, Asset Management",11.69B
1,RDY,"Healthcare, Drug Manufacturers - Specialty & G...",11.67B
2,GMED,"Healthcare, Medical Devices",11.67B


### Step 3: Data Type Conversion

This multi-part step cleans and converts all string-based numeric and percentage columns into proper numeric types.

#### Part A: Convert Abbreviated Currency Columns to Millions

In [4]:
def convert_to_millions(value: str) -> float:
    """Converts a string with a T/B/M/K suffix to a numeric value in millions."""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip().upper()
    if not value_str:
        return np.nan

    multipliers = {'T': 1_000_000, 'B': 1_000, 'M': 1, 'K': 0.001}
    suffix = value_str[-1]
    
    if suffix in multipliers:
        number_part = value_str[:-1]
        try:
            return float(number_part) * multipliers[suffix]
        except (ValueError, TypeError):
            return np.nan
    return np.nan

if df is not None:
    print("\n--- Step 3a: Converting currency columns to millions ---")
    new_names = {}
    for col in CURRENCY_COLS:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_millions)
            new_names[col] = f"{col}, M"
    
    df.rename(columns=new_names, inplace=True)
    print(f"Converted and renamed {len(new_names)} columns.")
    display(df[[name for name in new_names.values() if name in df.columns]].head(3))


--- Step 3a: Converting currency columns to millions ---
Converted and renamed 12 columns.


,"Market Cap, M","AUM, M","Sales, M","Income, M","Outstanding, M","Float, M","Short Interest, M","Avg Volume, M","Flows 1M, M","Flows 3M, M","Flows YTD, M","MktCap AUM, M"
0,11690.0,NaN,6290.0,669.20,445.09,442.08,12.76,5.15,NaN,NaN,NaN,11690.0
1,11670.0,NaN,3980.0,680.48,834.61,832.46,9.21,1.07,NaN,NaN,NaN,11670.0
2,11670.0,NaN,2770.0,423.78,112.18,111.38,4.40,1.73,NaN,NaN,NaN,11670.0


#### Part B: Convert Percentage Columns to Numeric

In [5]:
if df is not None:
    print("\n--- Step 3b: Converting percentage columns ---")
    percent_cols = [
        col for col in df.columns if df[col].dtype == 'object' and df[col].str.endswith('%', na=False).any()
    ]

    if not percent_cols:
        print("No new percentage columns found to modify.")
    else:
        print("Processing the following percentage columns:")
        for col in percent_cols:
            df[col] = pd.to_numeric(df[col].str.replace('%', ''), errors='coerce')
            new_name = f"{col} %" if '%' not in col else col
            df.rename(columns={col: new_name}, inplace=True)
            print(f"  - Converted '{col}' to numeric and renamed to '{new_name}'")


--- Step 3b: Converting percentage columns ---
Processing the following percentage columns:
  - Converted 'Dividend' to numeric and renamed to 'Dividend %'
  - Converted 'Payout Ratio' to numeric and renamed to 'Payout Ratio %'
  - Converted 'EPS This Y' to numeric and renamed to 'EPS This Y %'
  - Converted 'EPS Next Y' to numeric and renamed to 'EPS Next Y %'
  - Converted 'EPS Past 5Y' to numeric and renamed to 'EPS Past 5Y %'
  - Converted 'EPS Next 5Y' to numeric and renamed to 'EPS Next 5Y %'
  - Converted 'Sales Past 5Y' to numeric and renamed to 'Sales Past 5Y %'
  - Converted 'Sales Q/Q' to numeric and renamed to 'Sales Q/Q %'
  - Converted 'EPS Q/Q' to numeric and renamed to 'EPS Q/Q %'
  - Converted 'EPS YoY TTM' to numeric and renamed to 'EPS YoY TTM %'
  - Converted 'Sales YoY TTM' to numeric and renamed to 'Sales YoY TTM %'
  - Converted 'EPS Surprise' to numeric and renamed to 'EPS Surprise %'
  - Converted 'Revenue Surprise' to numeric and renamed to 'Revenue Surprise 

#### Part C: Convert Other String-Based Numeric Columns

In [6]:
if df is not None:
    print("\n--- Step 3c: Converting other numeric string columns ---")
    converted_count = 0
    for col in OTHER_NUMERIC_COLS:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].str.replace(',', '', regex=False), errors='coerce')
            converted_count += 1
            
    print(f"Converted {converted_count} additional columns to numeric type.")
    print("\nData types after all conversions:")
    df.info()


--- Step 3c: Converting other numeric string columns ---
Converted 32 additional columns to numeric type.

Data types after all conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 113 entries, No. to MktCap AUM, M
dtypes: float64(94), int64(2), object(17)
memory usage: 1.3+ MB


### Step 4: Final Processing - Sort, Index, and Rank

Sort the DataFrame by the unified liquidity metric, set the `Ticker` as the index, and add a final `Rank`.

In [7]:
if df is not None:
    print("\n--- Step 4: Finalizing DataFrame ---")
    
    # 1. Sort by the primary metric in descending order
    df.sort_values(by='MktCap AUM, M', ascending=False, inplace=True, na_position='last')
    print("Sorted DataFrame by 'MktCap AUM, M'.")
    
    # 2. Add a 'Rank' column based on the new sort order
    df['Rank'] = range(1, len(df) + 1)
    print("Added 'Rank' column.")
    
    # 3. Set 'Ticker' as the index
    if 'Ticker' in df.columns:
        df.set_index('Ticker', inplace=True)
        print("Set 'Ticker' as the index.")
    
    print("\nFinal DataFrame structure:")
    display(df[['Rank', 'Info', 'MktCap AUM, M']].head())


--- Step 4: Finalizing DataFrame ---
Sorted DataFrame by 'MktCap AUM, M'.
Added 'Rank' column.
Set 'Ticker' as the index.

Final DataFrame structure:


C:\Users\ping\AppData\Local\Temp\ipykernel_16908\3337454481.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Rank'] = range(1, len(df) + 1)


,Rank,Info,"MktCap AUM, M"
Ticker,,,
NVDA,1,"Technology, Semiconductors",4252990.0
AAPL,2,"Technology, Consumer Electronics",4111960.0
GOOGL,3,"Communication Services, Internet Content & Inf...",3738850.0
GOOG,4,"Communication Services, Internet Content & Inf...",3738850.0
MSFT,5,"Technology, Software - Infrastructure",3556620.0


### Step 5: Save and Verify Cleaned Data

Save the final, cleaned DataFrame to a new Parquet file and read it back to verify integrity.

In [8]:
if df is not None:
    print("\n--- Step 5: Saving and verifying data ---")
    try:
        # Ensure destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the file
        df.to_parquet(DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved cleaned data to: {DEST_PATH}")

        # Verify by loading it back
        loaded_df = pd.read_parquet(DEST_PATH, engine='pyarrow')
        print("\nVerification successful. First 20 rows of the saved file:")
        display(loaded_df.head(20))
        
    except Exception as e:
        print(f"An error occurred during save or verification: {e}")


--- Step 5: Saving and verifying data ---
Successfully saved cleaned data to: c:\Users\ping\Files_win10\python\py311\stocks\data\2025-12-12_df_finviz_stocks_etfs.parquet

Verification successful. First 20 rows of the saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS This Y %,EPS Next Y %,EPS Past 5Y %,EPS Next 5Y %,Sales Past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,4252990.0,43.35,23.01,0.48,22.73,35.78,70.17,55.00,4.89,2.49,0.02,0.04,12/4/2025,1.16,4.04,1.51,56.20,62.83,91.83,47.88,64.24,62.49,67.23,58.53,65.22,187140.0,99200.0,3.22,3.81,24300.00,23310.00,95.92,4.06,-0.99,68.18,0.02,1.00,1.23,232.90,77.15,107.36,77.27,4.47,3.71,0.08,0.09,70.05,58.84,53.01,-4.05,-9.69,-1.54,23.28,29.58,30.33,2.32,6.55,2.73,3.53,-3.80,-6.33,12.14,-17.52,3.23,-17.52,102.05,86.62 - 212.19,-17.52,524959.99,39.86,Nov 19/a,1/22/1999,Yes,Yes,36000.0,-3.35,0.08,1.33,190.03000,1.07,202419822,256.95,180.93,181.08,182.82,174.62,175.02,-3.27,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",4252990.0,1
AAPL,2,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,4111960.0,37.31,30.54,2.95,9.88,55.76,75.18,41.63,4.99,3.70,0.39,1.03,11/10/2025,13.66,7.46,2.67,10.48,10.56,17.91,10.36,8.71,7.94,91.14,22.85,6.43,416160.0,112010.0,4.10,0.23,14770.00,14760.00,99.92,0.10,-2.33,64.89,-0.15,0.88,2.63,129.46,30.93,171.42,68.44,0.89,0.86,1.22,1.52,46.91,31.97,26.92,-0.18,1.76,17.57,41.65,12.31,11.13,1.09,4.83,1.31,1.87,0.72,3.89,21.51,-3.58,14.05,-3.58,64.46,169.21 - 288.62,-3.58,437337.07,57.58,Oct 30/a,12/12/1980,Yes,Yes,166000.0,0.13,-0.04,2.04,49.22000,0.78,38442981,290.38,278.03,277.91,279.22,276.82,278.28,0.09,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Consumer Electronics",4111960.0,2
GOOGL,4,Alphabet Inc,"NDX, S&P 500",Communication Services,Internet Content & Information,USA,NASD,3738850.0,30.51,27.58,1.62,9.70,9.66,37.96,50.83,32.03,8.15,0.17,0.83,12/8/2025,7.46,10.14,2.63,32.01,5.68,26.76,17.07,16.73,16.21,35.35,34.26,13.55,385520.0,124250.0,26.79,2.41,5820.00,5800.00,99.68,51.94,0.02,38.75,-0.49,1.37,2.16,79.33,25.71,35.45,29.40,1.75,1.75,0.09,0.11,59.18,33.10,32.23,-3.73,7.88,22.92,77.07,67.03,63.39,1.08,9.26,2.83,3.04,0.23,9.81,48.09,-5.94,31.14,-5.94,120.09,140.53 - 328.83,-5.94,12779.97,55.60,Oct 29/a,8/19/2004,Yes,Yes,183323.0,-1.50,0.50,1.44,36.68000,0.92,33926673,334.08,312.43,314.00,314.87,305.56,309.29,-1.01,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Communication Services, Internet Content & Inf...",3738850.0,3
GOOG,3,Alphabet Inc,"NDX, S&P 500",Communication Services,Internet Content & Information,USA,NASD,3738850.0,30.63,27.68,1.62,9.70,9.69,37.96,50.83,32.03,8.18,0.19,0.83,12/8/2025,7.46,10.14,2.63,32.01,5.71,26.76,17.08,16.73,16.21,35.35,34.26,13.55,385520.0,124250.0,26.62,2.40,5420.00,5020.00,92.69,58.39,0.01,26.73,-0.49,0.83,1.78,41.66,25.71,35.45,29.40,1.75,1.75,0.09,0.11,59.18,33.10,32.23,-3.59,8.03,23.34,76.55,66.47,63.05,1.09,9.16,2.78,2.99,0.45,10.00,47.90,-5.52,31.20,-5.52,117